**Dataset**

In [1]:
pwd

'C:\\Users\\rp\\Desktop\\Transformers_course'

In [2]:
path='contracts_data/full_contract_txt/2ThemartComInc_19990826_10-12G_EX-10.10_6700288_EX-10.10_Co-Branding Agreement_ Agency Agreement.txt'

In [3]:
lines = open(path, 'r').readlines()

In [4]:
type(lines)

list

In [7]:
len(lines)

227

In [8]:
lines[0]

'CO-BRANDING AND ADVERTISING AGREEMENT\n'

In [9]:
lines[0].replace('\n', '')

'CO-BRANDING AND ADVERTISING AGREEMENT'

In [10]:
lines = [line.replace('\n', '') for line in lines]
lines[:5]

['CO-BRANDING AND ADVERTISING AGREEMENT',
 '',
 'THIS CO-BRANDING AND ADVERTISING AGREEMENT (the "Agreement") is made as of June 21, 1999 (the "Effective Date") by and between I-ESCROW, INC., with its principal place of business at 1730 S. Amphlett Blvd., Suite 233, San Mateo, California 94402 ("i-Escrow"), and 2THEMART.COM, INC. having its principal place of business at 18301 Von Karman Avenue, 7th Floor, Irvine, California 92612 ("2TheMart").',
 '',
 '1. DEFINITIONS.']

In [11]:
lines = [line for line in lines if len(line)>0]
lines

['CO-BRANDING AND ADVERTISING AGREEMENT',
 'THIS CO-BRANDING AND ADVERTISING AGREEMENT (the "Agreement") is made as of June 21, 1999 (the "Effective Date") by and between I-ESCROW, INC., with its principal place of business at 1730 S. Amphlett Blvd., Suite 233, San Mateo, California 94402 ("i-Escrow"), and 2THEMART.COM, INC. having its principal place of business at 18301 Von Karman Avenue, 7th Floor, Irvine, California 92612 ("2TheMart").',
 '1. DEFINITIONS.',
 '(a) "CONTENT" means all content or information, in any medium, provided by a party to the other party for use in conjunction with the performance of its obligations hereunder, including without limitation any text, music, sound, photographs, video, graphics, data or software. Content provided by 2TheMart is referred to herein as "2TheMart Content" and Content provided by i-Escrow is referred to herein as "i-Escrow Content."',
 '(b) "CO-BRANDED SITE" means the web-site accessible through Domain Name, for the Services implemente

**First Q&A model**

In [6]:
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore

C:\Users\rp\anaconda3\envs\ml\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [12]:
document_store = ElasticsearchDocumentStore(
    host='localhost',
    username='',
    password='',
    index='first_contract'
)

02/22/2022 18:38:15 - INFO - elasticsearch -   PUT http://localhost:9200/first_contract [status:200 request:1.274s]
02/22/2022 18:38:15 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.015s]


In [14]:
data_json = [
    {'text': line,
    'meta': {'source': 'contract'}} for line in lines
]

In [15]:
data_json

[{'text': 'CO-BRANDING AND ADVERTISING AGREEMENT',
  'meta': {'source': 'contract'}},
 {'text': 'THIS CO-BRANDING AND ADVERTISING AGREEMENT (the "Agreement") is made as of June 21, 1999 (the "Effective Date") by and between I-ESCROW, INC., with its principal place of business at 1730 S. Amphlett Blvd., Suite 233, San Mateo, California 94402 ("i-Escrow"), and 2THEMART.COM, INC. having its principal place of business at 18301 Von Karman Avenue, 7th Floor, Irvine, California 92612 ("2TheMart").',
  'meta': {'source': 'contract'}},
 {'text': '1. DEFINITIONS.', 'meta': {'source': 'contract'}},
 {'text': '(a) "CONTENT" means all content or information, in any medium, provided by a party to the other party for use in conjunction with the performance of its obligations hereunder, including without limitation any text, music, sound, photographs, video, graphics, data or software. Content provided by 2TheMart is referred to herein as "2TheMart Content" and Content provided by i-Escrow is referre

In [16]:
document_store.write_documents(data_json)

02/22/2022 18:40:26 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.969s]


In [18]:
import requests
requests.get('http://localhost:9200/first_contract/_count').json()

{'count': 98,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [36]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store)

In [37]:
from haystack.reader.farm import FARMReader
reader = FARMReader(model_name_or_path='deepset/bert-base-cased-squad2',
                   context_window_size=1500,
                   use_gpu=True)

02/22/2022 19:17:26 - INFO - farm.utils -   Using device: CPU 
02/22/2022 19:17:26 - INFO - farm.utils -   Number of GPUs: 0
02/22/2022 19:17:26 - INFO - farm.utils -   Distributed Training: False
02/22/2022 19:17:26 - INFO - farm.utils -   Automatic Mixed Precision: None
02/22/2022 19:17:45 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
02/22/2022 19:17:45 - INFO - farm.utils -   Using device: CPU 
02/22/2022 19:17:45 - INFO - farm.utils -   Number of GPUs: 0
02/22/2022 19:17:45 - INFO - farm.utils -   Distributed Training: False
02/22/2022 19:17:45 - INFO - farm.utils -   Automatic Mixed Precision: None
02/22/2022 19:17:46 - INFO - farm.infer -   Got ya 3 parallel workers to do inference ...
02/22/2022 19:17:46 - INFO - farm.infer -    0    0    0 
02/22/2022 19:17:46 - INFO - farm.infer -   /w\  /w\  /w\
02/22/2022 19:17:46 - INFO - farm.infer -   /'\  / \  /'\
02/22/2022 19:17:46 - INFO - farm.infer -       


In [38]:
from haystack.pipeline import ExtractiveQAPipeline
qa = ExtractiveQAPipeline(reader=reader, retriever=retriever)

In [55]:
qa.run(query='How is the agreement/contract called?')

02/22/2022 19:58:48 - INFO - elasticsearch -   POST http://localhost:9200/first_contract/_search [status:200 request:0.078s]
Inferencing Samples: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.13 Batches/s]


{'query': 'How is the agreement/contract called?',
 'no_ans_gap': 1.3431944847106934,
 'answers': [{'answer': 'ADVERTISING AGREEMENT',
   'score': 6.3878493309021,
   'probability': 0.6896494941760655,
   'context': 'THIS CO-BRANDING AND ADVERTISING AGREEMENT (the "Agreement") is made as of June 21, 1999 (the "Effective Date") by and between I-ESCROW, INC., with its principal place of business at 1730 S. Amphlett Blvd., Suite 233, San Mateo, California 94402 ("i-Escrow"), and 2THEMART.COM, INC. having its principal place of business at 18301 Von Karman Avenue, 7th Floor, Irvine, California 92612 ("2TheMart").',
   'offset_start': 21,
   'offset_end': 42,
   'offset_start_in_doc': 21,
   'offset_end_in_doc': 42,
   'document_id': '5338fc35-ea6e-42e9-bee9-492e97c984db',
   'meta': {'source': 'contract'}},
  {'answer': 'mutual written agreement of the parties prior to the conclusion of the term of this Agreement',
   'score': 5.171849250793457,
   'probability': 0.6562170617671402,
   'co

In [56]:
qa.run(query='When does the agreement/contract become effective?')

02/22/2022 19:59:13 - INFO - elasticsearch -   POST http://localhost:9200/first_contract/_search [status:200 request:0.033s]
Inferencing Samples: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.11 Batches/s]


{'query': 'When does the agreement/contract become effective?',
 'no_ans_gap': 9.40023410320282,
 'answers': [{'answer': 'June 21, 1999',
   'score': 8.823537826538086,
   'probability': 0.7508109845386354,
   'context': 'THIS CO-BRANDING AND ADVERTISING AGREEMENT (the "Agreement") is made as of June 21, 1999 (the "Effective Date") by and between I-ESCROW, INC., with its principal place of business at 1730 S. Amphlett Blvd., Suite 233, San Mateo, California 94402 ("i-Escrow"), and 2THEMART.COM, INC. having its principal place of business at 18301 Von Karman Avenue, 7th Floor, Irvine, California 92612 ("2TheMart").',
   'offset_start': 75,
   'offset_end': 88,
   'offset_start_in_doc': 75,
   'offset_end_in_doc': 88,
   'document_id': '5338fc35-ea6e-42e9-bee9-492e97c984db',
   'meta': {'source': 'contract'}},
  {'answer': 'one (1) year following the Launch Date',
   'score': 8.069137573242188,
   'probability': 0.7327543408784235,
   'context': '8.1 TERM.  The term of this Agreement sha

In [57]:
qa.run(query='When does the agreement/contract expire?')

02/22/2022 19:59:41 - INFO - elasticsearch -   POST http://localhost:9200/first_contract/_search [status:200 request:0.050s]
Inferencing Samples: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.30 Batches/s]


{'query': 'When does the agreement/contract expire?',
 'no_ans_gap': 0.3704953193664551,
 'answers': [{'answer': 'one (1) year following the Launch Date',
   'score': 6.220602512359619,
   'probability': 0.6851573034795813,
   'context': '8.1 TERM.  The term of this Agreement shall continue for one (1) year following the Launch Date, unless earlier terminated as provided herein. This Agreement may be renewed for any number of successive one (1) year terms by mutual written agreement of the parties prior to the conclusion of the term of this Agreement.  A party wishing to renew this Agreement shall give the other party notice thereof no less than thirty (30) days before the expiration of the term then in effect.  In the event that either party does not give such notice, the term of this Agreement shall be automatically renewed for another one (1) year.',
   'offset_start': 57,
   'offset_end': 95,
   'offset_start_in_doc': 57,
   'offset_end_in_doc': 95,
   'document_id': 'd138d21b-35a2

In [58]:
qa.run(query='What does the launch date mean?')

02/22/2022 20:00:07 - INFO - elasticsearch -   POST http://localhost:9200/first_contract/_search [status:200 request:0.072s]
Inferencing Samples: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.61s/ Batches]


{'query': 'What does the launch date mean?',
 'no_ans_gap': -2.363508701324463,
 'answers': [{'answer': '6/11/99',
   'score': 4.895197868347168,
   'probability': 0.6483740350949364,
   'context': 'By:/s/Dominic J. Magliarditi                By:/s/Sanjay Bajaj Name: Dominic J. Magliarditi                Name: Sanjay Bajaj Title: President                            Title: VP Business Development Date: 6/21/99                               Date: 6/11/99',
   'offset_start': 251,
   'offset_end': 258,
   'offset_start_in_doc': 251,
   'offset_end_in_doc': 258,
   'document_id': 'f0cb8fb7-f9b0-4b95-91d6-706e21f8fb1d',
   'meta': {'source': 'contract'}},
  {'answer': 'Effective Date',
   'score': -0.02014857530593872,
   'probability': 0.49937035735451873,
   'context': '2.3 LAUNCH TIMING.  Each party shall use good faith and reasonable efforts to expeditiously develop the Co-Branded Pages and the Information Transfer Mechanism.  In the event that, after using such efforts, the Launch Dat

In [60]:
qa.run(query='What is the country law of the agreement/contract?')

02/22/2022 20:05:09 - INFO - elasticsearch -   POST http://localhost:9200/first_contract/_search [status:200 request:0.078s]
Inferencing Samples: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.11 Batches/s]


{'query': 'What is the country law of the agreement/contract?',
 'no_ans_gap': 1.946242332458496,
 'answers': [{'answer': 'California',
   'score': 10.071636199951172,
   'probability': 0.778846082579955,
   'context': '12.1 GOVERNING LAW.  This Agreement will be governed and construed in accordance with the laws of the State of California without giving effect to conflict of laws principles.  Both parties submit to personal jurisdiction in California and further agree that any cause of action arising under this Agreement shall be brought in a court in Orange County, California.',
   'offset_start': 111,
   'offset_end': 121,
   'offset_start_in_doc': 111,
   'offset_end_in_doc': 121,
   'document_id': '55e1d9f2-234f-46c5-89e7-23e930527a17',
   'meta': {'source': 'contract'}},
  {'answer': 'California Escrow Law',
   'score': 7.266396522521973,
   'probability': 0.7126520752292581,
   'context': "6.7  SERVICE STANDARDS. i-Escrow will comply with all laws and regulations and act as an I

In [61]:
qa.run(query='What is the jurisdiction of the agreement/contract?')

02/22/2022 20:07:07 - INFO - elasticsearch -   POST http://localhost:9200/first_contract/_search [status:200 request:0.032s]
Inferencing Samples: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.06 Batches/s]


{'query': 'What is the jurisdiction of the agreement/contract?',
 'no_ans_gap': 3.105253219604492,
 'answers': [{'answer': 'Orange County, California',
   'score': 7.354029655456543,
   'probability': 0.7148900189438696,
   'context': '12.1 GOVERNING LAW.  This Agreement will be governed and construed in accordance with the laws of the State of California without giving effect to conflict of laws principles.  Both parties submit to personal jurisdiction in California and further agree that any cause of action arising under this Agreement shall be brought in a court in Orange County, California.',
   'offset_start': 339,
   'offset_end': 364,
   'offset_start_in_doc': 339,
   'offset_end_in_doc': 364,
   'document_id': '55e1d9f2-234f-46c5-89e7-23e930527a17',
   'meta': {'source': 'contract'}},
  {'answer': 'mutual written agreement of the parties',
   'score': 6.720517158508301,
   'probability': 0.6984788307727582,
   'context': '8.1 TERM.  The term of this Agreement shall continue for

**Second Q&A model**

In [26]:
from haystack.retriever.dense import DensePassageRetriever
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model='facebook/dpr-question_encoder-single-nq-base',
    passage_embedding_model='facebook/dpr-ctx_encoder-single-nq-base',
    use_gpu=True,
    embed_title=True
)

In [27]:
document_store.update_embeddings(retriever=retriever)

02/22/2022 18:50:52 - INFO - elasticsearch -   POST http://localhost:9200/first_contract/_count [status:200 request:0.042s]
02/22/2022 18:50:52 - INFO - haystack.document_store.elasticsearch -   Updating embeddings for 98 docs ...
02/22/2022 18:50:52 - INFO - elasticsearch -   POST http://localhost:9200/first_contract/_search?scroll=1d&size=10000 [status:200 request:0.069s]
02/22/2022 18:50:52 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll [status:200 request:0.021s]
02/22/2022 18:50:52 - INFO - elasticsearch -   DELETE http://localhost:9200/_search/scroll [status:200 request:0.067s]
Creating Embeddings: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [02:14<00:00, 19.23s/ Batches]
02/22/2022 18:53:09 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:2.532s]


In [28]:
from haystack.reader.farm import FARMReader
reader = FARMReader(model_name_or_path='deepset/bert-base-cased-squad2', use_gpu=True)

02/22/2022 18:53:53 - INFO - farm.utils -   Using device: CPU 
02/22/2022 18:53:53 - INFO - farm.utils -   Number of GPUs: 0
02/22/2022 18:53:53 - INFO - farm.utils -   Distributed Training: False
02/22/2022 18:53:53 - INFO - farm.utils -   Automatic Mixed Precision: None
02/22/2022 18:54:15 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
02/22/2022 18:54:15 - INFO - farm.utils -   Using device: CPU 
02/22/2022 18:54:15 - INFO - farm.utils -   Number of GPUs: 0
02/22/2022 18:54:15 - INFO - farm.utils -   Distributed Training: False
02/22/2022 18:54:15 - INFO - farm.utils -   Automatic Mixed Precision: None
02/22/2022 18:54:15 - INFO - farm.infer -   Got ya 3 parallel workers to do inference ...
02/22/2022 18:54:15 - INFO - farm.infer -    0    0    0 
02/22/2022 18:54:15 - INFO - farm.infer -   /w\  /w\  /w\
02/22/2022 18:54:15 - INFO - farm.infer -   /'\  / \  /'\
02/22/2022 18:54:15 - INFO - farm.infer -       


In [29]:
qa=ExtractiveQAPipeline(reader=reader, retriever=retriever)

In [32]:
qa.run(query='When does the contract become effective?')

Creating Embeddings: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.93 Batches/s]
02/22/2022 19:02:44 - INFO - elasticsearch -   POST http://localhost:9200/first_contract/_search [status:200 request:0.079s]
Inferencing Samples: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.04 Batches/s]


{'query': 'When does the contract become effective?',
 'no_ans_gap': 2.4711995124816895,
 'answers': [{'answer': 'one (1) year following the Launch Date',
   'score': 7.980982780456543,
   'probability': 0.730590945459921,
   'context': '.1 TERM.  The term of this Agreement shall continue for one (1) year following the Launch Date, unless earlier terminated as provided herein. This Agr',
   'offset_start': 56,
   'offset_end': 94,
   'offset_start_in_doc': 57,
   'offset_end_in_doc': 95,
   'document_id': 'd138d21b-35a2-4764-b795-317e8acfaafe',
   'meta': {'source': 'contract'}},
  {'answer': 'Upon expiration or termination of this Agreement, within fourteen (14) days',
   'score': 6.447325706481934,
   'probability': 0.6912384853486513,
   'context': "osed by a court or government agency. Upon expiration or termination of this Agreement, within fourteen (14) days of the other party's request, each p",
   'offset_start': 38,
   'offset_end': 113,
   'offset_start_in_doc': 831,
   'offs

In [33]:
qa.run(query='When does the contract expire?')

Creating Embeddings: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.52 Batches/s]
02/22/2022 19:03:17 - INFO - elasticsearch -   POST http://localhost:9200/first_contract/_search [status:200 request:0.084s]
Inferencing Samples: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.12 Batches/s]


{'query': 'When does the contract expire?',
 'no_ans_gap': 1.0652928352355957,
 'answers': [{'answer': 'within fourteen (14) days',
   'score': 10.313533782958984,
   'probability': 0.7840103484667197,
   'context': "ment agency. Upon expiration or termination of this Agreement, within fourteen (14) days of the other party's request, each party will return all Conf",
   'offset_start': 63,
   'offset_end': 88,
   'offset_start_in_doc': 881,
   'offset_end_in_doc': 906,
   'document_id': '188ccaf8-ade2-43a1-b4ae-18f2ec0e5e13',
   'meta': {'source': 'contract'}},
  {'answer': 'one (1) year following the Launch Date',
   'score': 7.533759117126465,
   'probability': 0.7194469361596557,
   'context': '.1 TERM.  The term of this Agreement shall continue for one (1) year following the Launch Date, unless earlier terminated as provided herein. This Agr',
   'offset_start': 56,
   'offset_end': 94,
   'offset_start_in_doc': 57,
   'offset_end_in_doc': 95,
   'document_id': 'd138d21b-35a2-4764-

In [34]:
qa.run(query='What is the launch date?')

Creating Embeddings: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.26 Batches/s]
02/22/2022 19:03:56 - INFO - elasticsearch -   POST http://localhost:9200/first_contract/_search [status:200 request:0.092s]
Inferencing Samples: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.15 Batches/s]


{'query': 'What is the launch date?',
 'no_ans_gap': -3.611757755279541,
 'answers': [{'answer': 'June 21, 1999',
   'score': 5.769726276397705,
   'probability': 0.672874684910556,
   'context': 'O-BRANDING AND ADVERTISING AGREEMENT (the "Agreement") is made as of June 21, 1999 (the "Effective Date") by and between I-ESCROW, INC., with its prin',
   'offset_start': 69,
   'offset_end': 82,
   'offset_start_in_doc': 75,
   'offset_end_in_doc': 88,
   'document_id': '5338fc35-ea6e-42e9-bee9-492e97c984db',
   'meta': {'source': 'contract'}},
  {'answer': '12.3 PUBLICITY',
   'score': -0.9841816425323486,
   'probability': 0.46928305459003655,
   'context': '12.3 PUBLICITY.  Prior to the release of any press releases or other similar promotional materials related to this Agreement, the releasing party shal',
   'offset_start': 0,
   'offset_end': 14,
   'offset_start_in_doc': 0,
   'offset_end_in_doc': 14,
   'document_id': 'db91e27f-0fa1-404e-bf84-5c4bff6ba214',
   'meta': {'source': 'co

In [35]:
qa.run(query='What is the applicable law?')

Creating Embeddings: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.07 Batches/s]
02/22/2022 19:16:06 - INFO - elasticsearch -   POST http://localhost:9200/first_contract/_search [status:200 request:0.092s]
Inferencing Samples: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.06 Batches/s]


{'query': 'What is the applicable law?',
 'no_ans_gap': -1.3745801448822021,
 'answers': [{'answer': 'same shall be and shall remain in full force and effect',
   'score': 4.6385722160339355,
   'probability': 0.6410263386669962,
   'context': 'same shall be and shall remain in full force and effect. Any waiver of any right of a party or any obligation of the other party hereunder must be mad',
   'offset_start': 0,
   'offset_end': 55,
   'offset_start_in_doc': 0,
   'offset_end_in_doc': 55,
   'document_id': '6850cbd8-5645-4011-bbb9-11ae55c38a63',
   'meta': {'source': 'contract'}},
  {'answer': 'waiver or relinquishment',
   'score': 3.697842836380005,
   'probability': 0.61354314468094,
   'context': 'orce performance of any obligation hereunder shall be deemed a waiver or relinquishment to any extent of that or any other right or obligation, in tha',
   'offset_start': 63,
   'offset_end': 87,
   'offset_start_in_doc': 167,
   'offset_end_in_doc': 191,
   'document_id': 'df6c2b12-